In [1]:
##imports 

import cv2
import joblib 
import pytesseract
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix


In [2]:
##Needed CSV's
trainingData = pd.read_csv("train-data.csv", index_col=0) 
testingData  = pd.read_csv("test.csv", index_col=0) 

In [3]:
##batch size (started with a small no. to train the algorithm on)
batch_size = 10000

In [4]:
##vectorizing the data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(trainingData[:batch_size]['OCR text'])

In [5]:
y = trainingData[:batch_size]["Label"]

In [8]:
print(X)

(320000, 3)

In [29]:
print(vectorizer.get_feature_names())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
print(X.shape)


(10000, 233180)


In [6]:
##Creating pipelines for Neural Networks

from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
mlp_pipeline = Pipeline([
    ('nn', MLPClassifier(max_iter=10000))
])

In [ ]:
##Listing the testing tuning parameters

import numpy as np
from sklearn.model_selection import GridSearchCV
grid_params = {
  'nn__learning_rate_init': np.linspace(0.001, 0.005, 1),
  'nn__activation':  ["relu", "logistic", "identity"],
  'nn__solver':  ["lbfgs", "sgd", "adam"],
}
clf = GridSearchCV(mlp_pipeline, grid_params, verbose=10)
clf.fit(X, y)
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs 
[CV]  nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs, score=0.637, total=15.2min
[CV] nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs 
[CV]  nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs, score=0.636, total=13.2min
[CV] nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs 
[CV]  nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs, score=0.672, total=12.4min
[CV] nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs 
[CV]  nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs, score=0.658, total=23.1min
[CV] nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs 
[CV]  nn__activation=relu, nn__learning_rate_init=0.001, nn__solver=lbfgs, score=0.627, total=21.5min
[CV] nn__activation=relu, nn__learning_rate_init=0.001, nn__

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 15.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 28.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 40.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 63.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 85.4min remaining:    0.0s


In [23]:
##Fiting the data and training it - modify tuning parameters here based on the GridsearchCV

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(max_iter=10000, verbose=True)

clf.fit(X, y)


C:\Users\coste\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:587: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(max_iter=10000, verbose=True)

In [13]:
##Testing data based on the fitted model

from tqdm import tqdm

#Prediction and accuracy
from sklearn.metrics import accuracy_score
documents1 = trainingData[:]["OCR text"]
y_true = trainingData[:]['Label']
input = vectorizer.transform(documents1)
prediction1= clf.predict(input)

dataframe = pd.DataFrame(list(y_true),prediction1)
index = 0
count = 0
for i in tqdm(list(y_true)):
    if i == prediction1[index]:
        count += 1
    index+=1
    
    
print("Accuracy:",accuracy_score(list(y_true),prediction1))
print(dataframe)

KeyboardInterrupt: 